In [ ]:
%load_ext autoreload
%autoreload 2
%cd ..

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import pickle
from typing import Dict, Tuple
from collections import Counter
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib.dates import date2num
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import scipy.signal
import scipy.fftpack

from util.paths import DATA_PATH
from util.datasets import SlidingWindowDataset, read_physionet_dataset, RespiratoryEventType, GroundTruthClass
from util.filter import apply_butterworth_bandpass_filter, apply_butterworth_lowpass_filter
from util.mathutil import get_peaks, PeakType, cluster_1d, IntRange
from rule_based import detect_respiratory_events
from rule_based.detector import _detect_airflow_resp_events
from util.event_based_metrics import OverlapsBasedConfusionMatrix, get_overlaps
from util.mathutil import normalize_robust
from ai_based.data_handling.ai_datasets import AiDataset

# Makes numpy raise errors instead of outputting warnings
np.seterr('raise')

# Some preparations to pretty-print tensors & ndarrays
np.set_printoptions(edgeitems=10)
np.core.arrayprint._line_width = 400

In [ ]:
dataset_folder = DATA_PATH / "tr04-0808"
config = SlidingWindowDataset.Config(
    downsample_frequency_hz=5,
    time_window_size=pd.to_timedelta("5 minutes"),
    time_window_stride=5,
    ground_truth_vector_width=1
)
sliding_window_dataset = SlidingWindowDataset(config=config, dataset_folder=dataset_folder, allow_caching=True)

print(f"#Physionet dataset samples: {len(sliding_window_dataset.signals)}")
print(f"#Sliding window positions: {len(sliding_window_dataset)}")
print(f"Timeframe of sliding window positions: {sliding_window_dataset.valid_center_points[-1] - sliding_window_dataset.valid_center_points[0]}")
print(f"Respiratory events list present: {sliding_window_dataset.respiratory_events is not None}")

### Outputting some statistics on the annotated respiratory events

In [ ]:
annotated_respiratory_events = sliding_window_dataset.respiratory_events

respiratory_event_type_counter = Counter([e.event_type for e in annotated_respiratory_events])
print("Respiratory event types as per annotations:")
print(" - " + "\n - ".join(f"{klass.name}: {cnt}" for klass, cnt in respiratory_event_type_counter.items()))
print()
print(f"{len(annotated_respiratory_events)} annotated respiratory events:")
print(" - " + "\n - ".join([f"#{i}: {evt}" for i, evt in enumerate(annotated_respiratory_events)]))

# Enrich whole sliding window dataset by "is awake" row
awake_series = sliding_window_dataset.awake_series
sliding_window_dataset.signals[awake_series.name] = awake_series
del awake_series

# Enrich whole sliding window dataset by an events outline
annotated_events_outline_mat = np.zeros(shape=(len(sliding_window_dataset.signals),))
for event in annotated_respiratory_events:
    start_idx = sliding_window_dataset.signals.index.get_loc(event.start, method="nearest")
    end_idx = sliding_window_dataset.signals.index.get_loc(event.end, method="nearest")
    annotated_events_outline_mat[start_idx:end_idx] = 1
annotated_events_outline_series = pd.Series(data=annotated_events_outline_mat, index=sliding_window_dataset.signals.index)
sliding_window_dataset.signals["Annotated respiratory events"] = annotated_events_outline_series

del annotated_events_outline_series, annotated_events_outline_mat


In [ ]:
event_num = 6
event = annotated_respiratory_events[event_num]
print(f"Duration of chosen respiratory event #{event_num}: {(event.end-event.start).total_seconds():.1f}s")
print(f"Type of chosen respiratory event: {event.event_type.name}")

window_center_point = event.start + (event.end-event.start)/2
window_data = sliding_window_dataset.get(center_point=window_center_point)
axes = window_data.signals.plot(figsize=(20, 13), subplots=True)

### Peak detection experiments

In [ ]:
event_num = 0
signal_name = "AIRFLOW"
event = annotated_respiratory_events[event_num]

window_center_point = event.start + (event.end-event.start)/2
window_data = sliding_window_dataset.get(center_point=window_center_point)

#####

kernel_width = int(sliding_window_dataset.config.downsample_frequency_hz*0.7)
peaks = get_peaks(waveform=window_data.signals[signal_name].values, filter_kernel_width=kernel_width)
peaks_mat = np.zeros(shape=(window_data.signals.shape[0],))
for p in peaks:
    peaks_mat[p.start:p.end] = p.extreme_value
peaks_ser = pd.Series(peaks_mat, index = window_data.signals.index, name=f"{signal_name} peaks")

#####

data = pd.concat([window_data.signals[signal_name], peaks_ser, window_data.signals["Annotated respiratory events"]], axis=1).fillna(method="pad")
data.plot(figsize=(20,7), subplots=False)

overall_baseline = np.sqrt(np.mean([np.square(p.extreme_value) for p in peaks]))
# overall_baseline = np.sqrt(np.median([np.square(p.extreme_value) for p in peaks]))
plt.axhline(y=overall_baseline, linestyle='--', color="pink")
plt.axhline(y=-overall_baseline, linestyle='--', color="pink")